In [3]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import numpy as np
import os
from tensorflow import keras
from  shutil import copyfile
import random
import zipfile

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

In [3]:
train_image_generator = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_image_generator = ImageDataGenerator(rescale=1./255)

In [4]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=32,
                                                     directory="./weather_dataset/train",
                                                     shuffle=True,
                                                     target_size=(128, 128),
                                                     class_mode='categorical'
                                                     )

Found 6535 images belonging to 4 classes.


In [5]:
test_generator = test_image_generator.flow_from_directory(
                                                  directory="./weather_dataset/test",
                                                  target_size=(128, 128),
                                                  batch_size=32,
                                                  class_mode='categorical')

Found 215 images belonging to 4 classes.


In [6]:
from tensorflow.keras.applications.vgg16 import VGG16

In [7]:
base_model = VGG16(input_shape = (128, 128, 3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')

2023-12-05 22:57:21.996039: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64:/opt/sw/other/apps/cuda/12.2.0/lib64:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/openmpi-4.1.2-4a/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/python-3.9.9-jh/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/tcl-8.6.11-d4/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/sqlite-3.37.1-6s/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/zlib-1.2.11-2y/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-9.3.0/gcc-10.3.0-ya/lib64:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-9.3.0/gcc-10.3.0-ya/lib
2023-12-05 22:57:21.996074: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would lik

In [8]:
for layer in base_model.layers:
    layer.trainable = False

In [9]:
from tensorflow.keras.layers import Dense, Flatten , GlobalAveragePooling2D

In [10]:
from tensorflow.keras.models import Model

In [11]:
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)

In [12]:
model_pre = Model(inputs=base_model.input, outputs=predictions)

In [13]:
model_pre.compile(optimizer=keras.optimizers.Adam(0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [14]:
his = model_pre.fit(train_data_gen,validation_data=test_generator, epochs=30)

Epoch 1/30
205/205 [==============================] - 88s 427ms/step - loss: 0.7001 - accuracy: 0.7252 - val_loss: 0.4992 - val_accuracy: 0.8140
Epoch 2/30
205/205 [==============================] - 87s 422ms/step - loss: 0.5255 - accuracy: 0.7940 - val_loss: 0.4659 - val_accuracy: 0.8326
Epoch 3/30
205/205 [==============================] - 87s 422ms/step - loss: 0.4947 - accuracy: 0.8080 - val_loss: 0.4452 - val_accuracy: 0.8326
Epoch 4/30
205/205 [==============================] - 86s 421ms/step - loss: 0.4565 - accuracy: 0.8260 - val_loss: 0.3943 - val_accuracy: 0.8651
Epoch 5/30
205/205 [==============================] - 87s 422ms/step - loss: 0.4497 - accuracy: 0.8248 - val_loss: 0.3738 - val_accuracy: 0.8744
Epoch 6/30
205/205 [==============================] - 86s 419ms/step - loss: 0.4273 - accuracy: 0.8346 - val_loss: 0.3874 - val_accuracy: 0.8744
Epoch 7/30
205/205 [==============================] - 87s 422ms/step - loss: 0.4114 - accuracy: 0.8409 - val_loss: 0.3483 - val_ac

In [10]:
eval_gen = ImageDataGenerator(rescale=1./255)

In [20]:
eval_images = eval_gen.flow_from_directory(
                                                  directory="./temp/",
                                                  target_size=(256, 256),
                                                  batch_size=32,
                                                  class_mode='categorical')

Found 28 images belonging to 4 classes.


In [17]:
eval_images.classes

array([], dtype=int32)

In [38]:
pred1 = model_pre.evaluate(eval_images)

3/3 [==============================] - 4s 1s/step - loss: 0.4879 - accuracy: 0.8378


In [18]:
from tensorflow.keras.applications.resnet import ResNet50

In [19]:
base_model_res = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

94781440/94765736 [==============================] - 0s 0us/step


In [20]:
for layer in base_model_res.layers:
    layer.trainable = False

In [21]:
x = GlobalAveragePooling2D()(base_model_res.output)
x = Dense(256, activation='relu')(x)
x = Dense(4, activation='softmax')(x)

In [24]:
model2 = Model(inputs=base_model_res.input, outputs=x)

# Compile the model
model2.compile(optimizer=keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/python-3.9.9-jh/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [26]:
model2.fit(train_data_gen,
          epochs=10,
          validation_data=test_generator)

Epoch 1/10
205/205 [==============================] - 44s 205ms/step - loss: 1.2393 - accuracy: 0.4239 - val_loss: 1.3715 - val_accuracy: 0.3209
Epoch 2/10
205/205 [==============================] - 42s 204ms/step - loss: 1.1131 - accuracy: 0.4939 - val_loss: 1.2824 - val_accuracy: 0.3721
Epoch 3/10
205/205 [==============================] - 41s 202ms/step - loss: 1.0477 - accuracy: 0.5634 - val_loss: 1.1434 - val_accuracy: 0.4884
Epoch 4/10
205/205 [==============================] - 41s 201ms/step - loss: 0.9978 - accuracy: 0.5802 - val_loss: 1.1157 - val_accuracy: 0.5163
Epoch 5/10
205/205 [==============================] - 41s 201ms/step - loss: 0.9577 - accuracy: 0.5986 - val_loss: 1.0492 - val_accuracy: 0.5907
Epoch 6/10
205/205 [==============================] - 41s 201ms/step - loss: 0.9283 - accuracy: 0.6186 - val_loss: 1.0050 - val_accuracy: 0.6093
Epoch 7/10
205/205 [==============================] - 42s 202ms/step - loss: 0.9101 - accuracy: 0.6263 - val_loss: 0.9797 - val_ac